In [25]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [27]:
df.shape

(28, 3)

In [28]:
from random import random
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

In [29]:
len(X_train)

22

In [30]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [31]:
X_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [32]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train_scaled,y_train,epochs=600)

Epoch 1/600
1/1 [==============================] - 0s 227ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/600
1/1 [==============================] - 0s 1ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/600
1/1 [==============================] - 0s 2ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/600
1/1 [==============================] - 0s 3ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/600
1/1 [=========

In [33]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 72ms/step - loss: 0.6135 - accuracy: 0.6667


[0.6134946346282959, 0.6666666865348816]

In [34]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [35]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 25ms/step


array([[0.6505675 ],
       [0.5947803 ],
       [0.43973374],
       [0.61437804],
       [0.65428597],
       [0.67619085]], dtype=float32)

In [36]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [37]:
coef,intercept = model.get_weights()
coef,intercept

(array([[0.8199031],
        [0.6915977]], dtype=float32),
 array([-0.45541763], dtype=float32))

In [38]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [39]:
def prediction_function(age,affordibility):
    weighted_sum = coef[0]*age+coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [40]:
prediction_function(.47,1)

0.6505674856837084

In [41]:
prediction_function(.18,1)

0.5947802743403883

In [42]:
def log_loss(y_true,y_predicted):
    epsilon= 1e-15
    y_predicted_new = [max(i,epsilon)for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon)for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [43]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [44]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 2

    def predict(self,X_test):
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def fit(self,X,y,epochs,loss_threshold):
        self.w1,self.w2,self.bias = self.gradient_descent(X['age'],X['affordibility'],y,epochs,loss_threshold)
        

    def gradient_descent(self,age, affordability, y_true, epochs,loss_threshold):
        # w1,w2,bias
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)

        for i in range(epochs):
            weighted_sum = w1*age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)

            loss = log_loss(y_true,y_predicted)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true))

            bias_d = np.mean(y_predicted-y_true)

            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d

            bias = bias -rate * bias_d
            
            if i%50==0:
                print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_threshold:
                print(f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break
        return w1,w2,bias
    
    def sigmoid_numpy(X):
        return 1/(1+np.exp(-X))

    def log_loss(y_true,y_predicted):
        epsilon= 1e-15
        y_predicted_new = [max(i,epsilon)for i in y_predicted]
        y_predicted_new = [min(i,1-epsilon)for i in y_predicted_new]
        y_predicted_new = np.array(y_predicted_new)
        return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [141]:
customModel= myNN()
customModel.fit(X_train_scaled,y_train,epochs=500,loss_threshold=0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:50, w1:1.503319554173139, w2:1.108384790367645, bias:-1.2319047301235464, loss:0.5675865113475955
Epoch:100, w1:2.200713131760032, w2:1.2941584023238903, bias:-1.6607009122062801, loss:0.5390680417774752
Epoch:150, w1:2.8495727769689085, w2:1.3696895491572745, bias:-1.986105845859897, loss:0.5176462164249294
Epoch:200, w1:3.443016970881803, w2:1.4042218624465033, bias:-2.2571369883752723, loss:0.5005011269691375
Epoch:250, w1:3.982450494649576, w2:1.4239127329321233, bias:-2.494377365971801, loss:0.48654089537617085
Epoch:300, w1:4.472179522095915, w2:1.438787986553552, bias:-2.707387811922373, loss:0.4750814640632793
Epoch:350, w1:4.917245868007634, w2:1.4525660781176122, bias:-2.901176333556766, loss:0.46561475306999006
Epoch:366, w1:5.051047623653049, w2:1.4569794548473887, bias:-2.9596534546250037, loss:0.46293944095888917


In [139]:
customModel.predict(X_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64

In [140]:

weights = np.array([[5.060867],[1.4086502]])
bias = np.array([-2.9137027])



model.set_weights([weights,bias])

In [137]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 14ms/step


array([[0.70548487],
       [0.35569552],
       [0.16827849],
       [0.47801173],
       [0.7260697 ],
       [0.82949835]], dtype=float32)

ValueError: Layer sequential_1 weight shape (2, 1) is not compatible with provided weight shape ().